In [1]:
import numpy as np

In [2]:

def initialize_parameters_deep(layer_dims):
    parameters = {}
    L = len(layer_dims)           

    for l in range(1, L):
        parameters['W' + str(l)] = np.random.randn(layer_dims[l],layer_dims[l-1])*0.01
        parameters['b' + str(l)] = np.zeros((layer_dims[l],1))
    return parameters

In [40]:

def linear_forward(A, W, b):
    Z = np.dot(W.T,A)+b
    cache = (A, W, b)
    
    return Z, cache

In [34]:
def sigmoid(z):
    a = 1/(1+np.exp(-z))
    return a

In [26]:
def relu(z):
    a = np.max(z,0)
    return a

In [28]:

def linear_activation_forward(A_prev, W, b, activation='sigmoid'):
    
    if activation == "sigmoid":
        Z, linear_cache = linear_forward(A_prev,W,b)
        A = sigmoid(Z)
    
    elif activation == "relu":
        Z, linear_cache = linear_forward(A_prev,W,b)
        A =  relu(Z)
        
    cache = (linear_cache, linear_cache)

    return A, cache

In [8]:
def L_model_forward(X, parameters):

    caches = []
    A = X
    L = len(parameters) // 2                  # number of layers in the neural network
    for l in range(1, L):
        A_prev = A 
        A, cache = linear_activation_forward(A_prev, parameters['W'+str(l)], parameters['b'+str(l)], activation='relu')
        caches.append(cache)
    
    AL, cache = linear_activation_forward(A, parameters['W'+str(l+1)], parameters['b'+str(l+1)], activation='sigmoid')
    caches.append(cache)
    
    return AL, caches

In [9]:

def compute_cost(AL, Y):
    
    m = Y.shape[1]

    cost = -np.sum(np.dot(Y,np.log(AL).T)+np.dot((1-Y),np.log(1-AL).T))/m
    
    cost = np.squeeze(cost)      # To make sure your cost's shape is what we expect (e.g. this turns [[17]] into 17).
    
    return cost

In [10]:

def linear_backward(dZ, cache):
    
    A_prev, W, b = cache
    m = A_prev.shape[1]


    dW = np.dot(dZ,A_prev.T)/m
    db = np.sum(dZ,axis=1,keepdims=True)/m
    dA_prev = np.dot(W.T,dZ)
    
    return dA_prev, dW, db

In [11]:
def linear_activation_backward(dA, cache, activation):
    linear_cache, activation_cache = cache
    
    if activation == "relu":
        dZ = relu_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
        
    elif activation == "sigmoid":
        dZ = sigmoid_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
        
    return dA_prev, dW, db

In [12]:
def L_model_backward(AL, Y, caches):
    
    grads = {}
    L = len(caches) 
    m = AL.shape[1]
    Y = Y.reshape(AL.shape) 
    
    dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL)) 
    
    
    current_cache = caches[L-1]
    grads["dA" + str(L)], grads["dW" + str(L)], grads["db" + str(L)] = linear_activation_backward(dAL, current_cache, activation = "sigmoid")
    
    
    for l in reversed(range(L-1)):
        
        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = linear_activation_backward(grads["dA" + str(l + 2)], current_cache, activation = "relu")
        
        grads["dA" + str(l + 1)] = dA_prev_temp
        grads["dW" + str(l + 1)] = dW_temp
        grads["db" + str(l + 1)] = db_temp
        
    return grads

In [13]:
def update_parameters(parameters, grads, learning_rate):
    
    L = len(parameters) // 2 # number of layers in the neural network

    for l in range(L):
        parameters["W" + str(l+1)] = parameters["W" + str(l+1)] - grads['dW'+str(l+1)]*learning_rate
        parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - grads['db'+str(l+1)]*learning_rate
        
    return parameters

In [14]:
def L_layer_model(X, Y, layers_dims, learning_rate = 0.0075, num_iterations = 3000, print_cost=False):
    
    
    costs = []                        
    
    parameters = initialize_parameters_deep(layers_dims)
    
    for i in range(0, num_iterations):

        AL, caches = L_model_forward(X, parameters)
        
        cost = compute_cost(AL, Y)
        
        grads = L_model_backward(AL, Y, caches)
        
        parameters = update_parameters(parameters, grads, learning_rate)
        
        if print_cost and i % 100 == 0:
            print ("Cost after iteration %i: %f" % (i, cost))
        if print_cost and i % 100 == 0:
            costs.append(cost)
            
    
    plt.plot(np.squeeze(costs))
    plt.ylabel('cost')
    plt.xlabel('iterations (per tens)')
    plt.title("Learning rate =" + str(learning_rate))
    plt.show()
    
    return parameters

In [22]:
import pandas as pd
data = pd.read_csv('data/mushrooms.csv')
from sklearn.preprocessing import LabelEncoder
ch = list(data.columns.values)
for i in ch:
    encoder = LabelEncoder()
    col = data[i]
    col = encoder.fit_transform(col)
    data[i]=col
x = data.drop('class', axis=1).values
y = data['class'].values
y = y.reshape((-1,1))
print(x.shape, y.shape)

(8124, 22) (8124, 1)


In [23]:
x = x.T
y = y.T
print(x.shape, y.shape)

(22, 8124) (1, 8124)


In [24]:
layers_dims = [22, 20, 7, 5, 1] #  5-layer model

In [41]:
parameters = L_layer_model(x, y, layers_dims, num_iterations = 2500, print_cost = True)

ValueError: shapes (22,20) and (22,8124) not aligned: 20 (dim 1) != 22 (dim 0)